In [1]:
import pandas as pd
import numpy as np
from __future__ import division
head = ['class', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution', 'physician-fee-freeze','el-salvador-aid','religious-groups-in-schools','anti-satellite-test-ban','aid-to-nicaraguan-contras','mx-missile','immigration','synfuels-corporation-cutback','education-spending','superfund-right-to-sue','crime','duty-free-exports', 'export-administration-act-south-africa']
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data", header=None, names=head)

In [2]:
data.columns.values

array(['class', 'handicapped-infants', 'water-project-cost-sharing',
       'adoption-of-the-budget-resolution', 'physician-fee-freeze',
       'el-salvador-aid', 'religious-groups-in-schools',
       'anti-satellite-test-ban', 'aid-to-nicaraguan-contras',
       'mx-missile', 'immigration', 'synfuels-corporation-cutback',
       'education-spending', 'superfund-right-to-sue', 'crime',
       'duty-free-exports', 'export-administration-act-south-africa'], dtype=object)

In [37]:
data.head()

,class,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [38]:
data.replace("?","-",inplace=True)

In [43]:
head[1:17]

['handicapped-infants',
 'water-project-cost-sharing',
 'adoption-of-the-budget-resolution',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups-in-schools',
 'anti-satellite-test-ban',
 'aid-to-nicaraguan-contras',
 'mx-missile',
 'immigration',
 'synfuels-corporation-cutback',
 'education-spending',
 'superfund-right-to-sue',
 'crime',
 'duty-free-exports',
 'export-administration-act-south-africa']

In [40]:
_values = data.values

In [41]:
#targets
Y = _values[:,0]
#train data
X = _values[:,1:17]

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [45]:
class treeClassifier(): 
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        return self
    
    def predict(self, X_test):
        pass
    
    def calc_entropy(self,p):
        if p!=0:
            return -p * np.log2(p)
        else:
            return 0
        
    
    def get_values(self,dataset, features):
        values = []
        for datapoint in dataset:
            if datapoint[feature] not in values:
                values.append(datapoint[feature])
                
        return values
    
    def cal_info_gain(self, feature):
        gain = 0
        nData = len(data)
        # List the values that feature can take
        values = []
        for datapoint in self.X_train:
            if datapoint[feature] not in values:
                values.append(datapoint[feature])
                
                
        featureCounts = np.zeros(len(values))
        entropy = np.zeros(len(values))
        valueIndex = 0
        
        for value in values:
            dataIndex = 0
            newClasses = []
            for datapoint in self.X_train:
                if datapoint[feature] == value:
                    featureCounts[valueIndex]+=1
                    newClasses.append(self.y_train[dataIndex])
                    
                dataIndex +=1
                
                
                classValues = []
                for aclass in newClasses:
                    if classValues.count(aclass) == 0:
                        classValues.append(aclass)
                classCounts = np.zeros(len(classValues))
                classIndex = 0
            for classValue in classValues:
                for aclass in newClasses:
                    if aclass == classValue:
                        classCounts[classIndex]+=1
                classIndex +=1
                    
            for classIndex in range(len(classValues)):
                entropy[valueIndex] += self.calc_entropy(float(classCounts[classIndex]))/sum(classCounts)
                
            gain += float(featureCounts[valueIndex]) / nData * entropy[valueIndex]
            valueIndex += 1
        return gain
                
        
    def make_tree(data, classes, featuresNames):
        the_set = set(classes);
        
        freq = np.zeros(len(the_set))
        
        index = 0
        for i in the_set:
            freq[i] = classes.count(i)
            index += 1
            
        default = classes[np.argmax(freq)]
        
        
        nData = len(data);
        nFeatures = len(featuresNames);
        
        if nData == 0 or nFeatures == 0:
            return default
        
        elif classes.count(classes[0]) == nData:
            return classes[0]
        else:
            gain = np.zeros(nFeatures)
            for feature in range(nFeatures):
                g = cal_info_gain(data, classes,feature)
                gain[feature] = totalEntropy - g
            
            bestFeature = np.argmax(gain)
            tree = {featuresNames[bestFeature]: {}}
            
            
            values = self.get_values(data, featuresNames)
            for value in values:
                for datapoint[bestFeature]==value:
                    if bestFeature == 0:
                        datapoint = datapoint[1:]
                        newNames = featuresNames[1:]
                    elif bestFeature == nFeatures:
                        datapoint = datapoint[:-1]
                        newNames = featuresNames[:-1]

                    else:
                        datapoint = datapoint[:bestFeature]
                        datapoint.extend(datapoint[bestFeature+1:])
                        newNames = featuresNames[:bestFeature]
                        newNames.extend(featuresNames[bestFeature+1:])

                    newData.append(datapoint)
                    newClasses.append(classes[index])
                index +=1
                
            subtree = make_tree(newData,newClasses,newNames)
            
            tree[featuresNames[bestFeature][value]] = subtree
        return tree
                
                
        

In [44]:
my_array = [{"A": 3}, {"B":2}]

myValue = my_array[np.argmax(i.key for i in my_array)]

NameError: name 'frequency' is not defined